d
<h2>Observações a cerca da ingestão de dados:</h2>
<ul>
<li>Como a tabela Microdados tem quase 2.0GB, tive algums problemas em relizar o upload de uma só vez, a solução foi splittar o arquivo em 12 partes.</li>
<li>A partir disso, consegui realizar o ingest das 12 tabelas.</li>
</ul>

<h2>Elaboração da ETL:</h2>
<h4>Tecnologias utilizadas:</h4>
<ul>
<li>Azure Databricks: ingest, transformação e processamento dos dados.</li>
<li>SQL</li>
<li>PySpark</li>
</ul>

<h4>Importando algumas bibliotecas:</h4>

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import datetime
import re
import json
import pandas as pd

#<h4>Nesso momento, iremos juntar as partes de todas as tabelas que fizemos o ingest. Criei uma view para alocar essa base:</h4>

In [0]:
%sql
CREATE OR REPLACE VIEW experimentos_comercial.e2_base AS 
SELECT * 
FROM experimentos_comercial.e_0
  UNION ALL
SELECT *
FROM experimentos_comercial.e_1
  UNION ALL
SELECT *
FROM experimentos_comercial.e_2
  UNION ALL
SELECT *
FROM experimentos_comercial.e_3
  UNION ALL
SELECT *
FROM experimentos_comercial.e_4
  UNION ALL
SELECT *
FROM experimentos_comercial.e_5
  UNION ALL
SELECT *
FROM experimentos_comercial.e_6
  UNION ALL
SELECT *
FROM experimentos_comercial.e_7
  UNION ALL
SELECT *
FROM experimentos_comercial.e_8
  UNION ALL
SELECT *
FROM experimentos_comercial.e_9
  UNION ALL
SELECT *
FROM experimentos_comercial.e_10
  UNION ALL
SELECT *
FROM experimentos_comercial.e_11


In [0]:
%sql
REFRESH TABLE  experimentos_comercial.e2_base

#<h4>Visualizando as primeiras linhas da view:</h4>

In [0]:
%sql
SELECT * 
FROM experimentos_comercial.e2_base
LIMIT 10

NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
200006267922,2020,2,F,1,1,1,2,0,3,1,0,3106200,Belo Horizonte,31,MG,4,1,1,3106200,Belo Horizonte,31,MG,1,1,1,1,597,569,580,587,597.9,642.2,525.3,534.3,DDCEDCBADECEBECDCCAACDDCBDACABECCDDAEADDECADA,AEDEADEADABBAECDDCDDCDADCEDDACEDACDBECBCDCEBD,BCDAE99999BBEEBECDBAAADACAEABCDDDDEDAAAEDEBBECEACA,DCDCCDEEACAACBCBBDECAEDAEBCEADCCACDEEBEABBCAC,0,DDBACBDDDECDAEDCCCEDBDDCBDDCAAECEDDACCBAEDCDX,DECCADEBDECBBECDDCBACDBDEEDABCDDAEDBECECBCCBD,DCEAAEEEACAACBBEBEBAAADBBAEBCEDBAADADDEADEBDACCAAA,DABCCAEDBBCCCABCACCBDXCCCBEEEDBBACCDACBAEBAEC,1,140,120,120,120,100,600,F,F,E,E,4,P,D,E,E,C,A,B,B,B,B,B,B,B,E,B,B,E,B,C,B
200002237066,2020,3,M,1,1,1,2,0,2,1,0,2707909,Santa Luzia do Norte,27,AL,2,1,1,2706901,Pilar,27,AL,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,E,E,B,B,4,B,A,B,C,A,B,B,A,A,A,A,A,A,B,A,A,C,A,A,A
200001457661,2020,3,M,1,3,1,2,0,2,1,0,3107802,Bom Jesus do Galho,31,MG,2,1,1,3113404,Caratinga,31,MG,1,1,1,1,599,568,578,590,490.5,460.7,558,617,EADCECCACDACDDDAABAEADEDEBECADEDDBCDADAADEEEC,BEECAABDAEAADAECDDDBAEEDCADBDBCBECDEBCBDDADEB,99999BAAECCEBAEADBCADBAEEEADEADBADACDCAAACBEDDEDBA,CABCCBCACECBAECEDCACAAEBEBDACCAEDCCEDDACADBBB,1,CBDDDDCAAECEDDCEDBDDAEDECAEDCCDDDCBDCDXBAACCB,BDECBCCDEDDAEBDCDDCBBEDABCCBACDBDEECCADEDBECE,CDEAACAEEEBEBADADAAAAACBAEBDEADACDBBDBACECAAADEBBE,CABCAACCACDABEEEBAECXCCCEEDBACCDBCCABBCCBDDBB,1,160,140,140,160,120,720,E,D,C,B,4,C,A,B,D,B,A,B,A,B,A,A,A,A,B,B,A,D,A,A,B
200001190770,2020,10,M,1,3,1,1,5,1,null,0,null,null,null,null,null,null,null,3205309,Vitória,32,ES,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
200002333375,2020,16,F,4,1,1,1,14,1,null,0,null,null,null,null,null,null,null,5002704,Campo Grande,50,MS,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,null,null,null,null,null,C,D,C,B,2,B,A,B,C,A,A,C,B,B,A,B,A,A,B,A,A,C,B,B,B
200006478999,2020,6,F,1,1,1,1,0,1,null,0,null,null,null,null,null,null,null,3304557,Rio de Janeiro,33,RJ,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,H,D,F,D,6,E,A,B,C,A,A,B,A,A,A,A,A,A,B,A,A,D,A,B,B
200001627853,2020,4,F,1,1,1,1,1,1,null,0,null,null,null,null,null,null,null,2402006,Caicó,24,RN,1,1,1,1,598,567,577,588,414.6,497.2,493.2,331.7,CAADBCCCBEADDAAEABEBAAEAAEDCEAAAEDECEBDCBCABE,AABDBBBAADEDBCACDEDBEADCAECAADEAABCCDADAEBEDD,99999CEAADDACBAAAEBCBAEDDBACDADBCCBEBADCDAACABDDBA,ADDDBAAACEAABECDEAEABEACDBEBEAAEDCDADECDEBBCE,1,CEDBDDDCACCBDAEBADCBDCDXAAECEDDDECDDCBDDAEDCC,DABCBDDDAEDBECECBBEDEBDEECBCCCBACDCCADEBDECDD,AAEDCACEEECEAACBAEBBEDACDBAADADDEBBEBAAADEACAAADBB,AEBCCXCCCACCDABCCABCABCBAECEDBBDBEEEDBBACCDAC,1,100,120,120,120,120,580,E,D,F,F,4,A,A,B,B,A,A,B,A,A,A,A,A,A,B,A,A,C,A,A,B
200002494273,2020,4,F,1,3,1,1,0,1,null,0,null,null,null,null,null,null,null,3541000,Praia Grande,35,SP,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,n

#<h4>Com base na viewm vamos construir um Data Frame. Seguiremos tembém alterando o DataType de algumas variáveis e selecionar apenas as colunas que nos interessam:</h4>

In [0]:
df_dados = spark.sql("""
SELECT 

DISTINCT NU_INSCRICAO,
CAST (NU_ANO AS INT),
CAST (TP_FAIXA_ETARIA AS INT),
TP_SEXO,
CAST (TP_COR_RACA AS INT),
CAST (TP_ESCOLA AS INT),
CAST (TP_ENSINO AS INT),
CAST (IN_TREINEIRO AS INT),
CO_MUNICIPIO_ESC,
NO_MUNICIPIO_ESC,
CO_UF_ESC,
SG_UF_ESC,
CAST (TP_DEPENDENCIA_ADM_ESC AS INT),
CAST (TP_LOCALIZACAO_ESC AS INT),
CAST (TP_SIT_FUNC_ESC AS INT),
CO_MUNICIPIO_PROVA,
NO_MUNICIPIO_PROVA,
CO_UF_PROVA,
SG_UF_PROVA,
CAST (TP_PRESENCA_CN AS INT),
CAST (TP_PRESENCA_CH AS INT),
CAST (TP_PRESENCA_LC AS INT),
CAST (TP_PRESENCA_MT AS INT),
CAST (NU_NOTA_CN AS DECIMAL(10,2)),
CAST (NU_NOTA_CH AS DECIMAL(10,2)),
CAST (NU_NOTA_LC AS DECIMAL(10,2)),
CAST (NU_NOTA_MT AS DECIMAL(10,2)),
CAST (TP_LINGUA AS INT),
CAST (TP_STATUS_REDACAO AS INT),
CAST (NU_NOTA_REDACAO AS DECIMAL(10,2))

FROM experimentos_comercial.e2_base

""")

#<h4>Verificando os datatypes das colunas:</h4>

In [0]:
print(df_dados.dtypes)

[('NU_INSCRICAO', 'string'), ('NU_ANO', 'int'), ('TP_FAIXA_ETARIA', 'int'), ('TP_SEXO', 'string'), ('TP_COR_RACA', 'int'), ('TP_ESCOLA', 'int'), ('TP_ENSINO', 'int'), ('IN_TREINEIRO', 'int'), ('CO_MUNICIPIO_ESC', 'string'), ('NO_MUNICIPIO_ESC', 'string'), ('CO_UF_ESC', 'string'), ('SG_UF_ESC', 'string'), ('TP_DEPENDENCIA_ADM_ESC', 'int'), ('TP_LOCALIZACAO_ESC', 'int'), ('TP_SIT_FUNC_ESC', 'int'), ('CO_MUNICIPIO_PROVA', 'string'), ('NO_MUNICIPIO_PROVA', 'string'), ('CO_UF_PROVA', 'string'), ('SG_UF_PROVA', 'string'), ('TP_PRESENCA_CN', 'int'), ('TP_PRESENCA_CH', 'int'), ('TP_PRESENCA_LC', 'int'), ('TP_PRESENCA_MT', 'int'), ('NU_NOTA_CN', 'decimal(10,2)'), ('NU_NOTA_CH', 'decimal(10,2)'), ('NU_NOTA_LC', 'decimal(10,2)'), ('NU_NOTA_MT', 'decimal(10,2)'), ('TP_LINGUA', 'int'), ('TP_STATUS_REDACAO', 'int'), ('NU_NOTA_REDACAO', 'decimal(10,2)')]

In [0]:
df_dados.agg(count("NU_INSCRICAO").alias("Total de linhas")).show()
df_dados.select(countDistinct("NU_INSCRICAO").alias("Total de inscritos")).show()

+---------------+
Total de linhas|
+---------------+
 5783109|
+---------------+

+------------------+
Total de inscritos|
+------------------+
 5783109|
+------------------+

#<h4>Visualizando as primeiras linhas do nosso data frame:</h4>

In [0]:
df_dados.limit(10).display()

NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_COR_RACA,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_REDACAO
200002237066,2020,3,M,1,2,1,0,2707909,Santa Luzia do Norte,27,AL,2,1,1,2706901,Pilar,27,AL,0,0,0,0,null,null,null,null,0,null,null
200003756009,2020,7,M,3,1,null,0,null,null,null,null,null,null,null,3301702,Duque de Caxias,33,RJ,1,1,1,1,451.60,532.70,514.30,433.20,1,1,300.00
200002333375,2020,16,F,1,1,null,0,null,null,null,null,null,null,null,5002704,Campo Grande,50,MS,0,0,0,0,null,null,null,null,1,null,null
200001457661,2020,3,M,3,2,1,0,3107802,Bom Jesus do Galho,31,MG,2,1,1,3113404,Caratinga,31,MG,1,1,1,1,490.50,460.70,558.00,617.00,1,1,720.00
200002494273,2020,4,F,3,1,null,0,null,null,null,null,null,null,null,3541000,Praia Grande,35,SP,0,0,0,0,null,null,null,null,0,null,null
200006643518,2020,7,F,3,1,null,0,null,null,null,null,null,null,null,2925303,Porto Seguro,29,BA,0,0,0,0,null,null,null,null,0,null,null
200006478999,2020,6,F,1,1,null,0,null,null,null,null,null,null,null,3304557,Rio de Janeiro,33,RJ,0,0,0,0,null,null,null,null,0,null,null
200006267922,2020,2,F,1,3,1,0,3106200,Belo Horizonte,31,MG,4,1,1,3106200,Belo Horizonte,31,MG,1,1,1,1,597.90,642.20,525.30,534.30,0,1,600.00
200001627853,2020,4,F,1,1,null,0,null,null,null,null,null,null,null,2402006,Caicó,24,RN,1,1,1,1,414.60,497.20,493.20,331.70,1,1,580.00
200001190770,2020,10,M,3,1,null,0,null,null,null,null,null,null,null,3205309,Vitória,32,ES,0,0,0,0,null,null,null,null,1,null,null


#<h2>Pré processamento dos dados:</h2>
#<h4>Adicionando uma coluna com a soma das notas</h4>
#<h4>Adicionando uma coluna com a média</h4>

In [0]:
df_dados = df_dados.withColumn("SOMA_NOTA", df_dados['NU_NOTA_CN'] + df_dados['NU_NOTA_CH'] + df_dados['NU_NOTA_LC'] + df_dados['NU_NOTA_MT'] + df_dados['NU_NOTA_REDACAO'] )

In [0]:
df_dados = df_dados.withColumn("MÉDIA", (col("SOMA_NOTA")/5).cast(DecimalType(10,2)) )

#<h4>Adicionando uma coluna com a etnia, baseada no código da coluna TP_COR_RACA. Podemos fazê-la de duas formas:</h4>
<ul>
<li>Através de um ingest de uma tabela auxiliar, buscando o valor com o método .join() com Pyspark.</li>
<li>Adicionando uma nova coluna condicional, com o método .when()</li>
</ul>
<h4>Nesse caso, iremos fazer com a segunda forma:</h4>

In [0]:

local = "/FileStore/tables/etnia.csv"
tipo = "csv"


infer_schema = "false"
first_row_is_header = "false"
delimiter = ";"

df = spark.read.format(tipo) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(local)

#Renomendo os cabeçalhos
df = df.withColumnRenamed("_c0","COD_ETNIA")\
       .withColumnRenamed("_c1","ETNIA")

#Visualizando
display(df)

COD_ETNIA,ETNIA
0,Não declarado
1,Branca
2,Preta
3,Parda
4,Amarela
5,Indígena


In [0]:
# Criando a tabela temporária

nome_tabela_temp = "etnia_csv"

df.createOrReplaceTempView(nome_tabela_temp)

In [0]:
%sql
/* Testando uma query */
select * from `etnia_csv`

COD_ETNIA,ETNIA
0,Não declarado
1,Branca
2,Preta
3,Parda
4,Amarela
5,Indígena


In [0]:
#Criando a tabela permanente

df.write.format("parquet").saveAsTable("experimentos_comercial.et_csv", mode="overwrite")

In [0]:
#Transformando a tabela em data frame
df_apoio_etnia = spark.sql("""
SELECT 
CAST (COD_ETNIA AS INT),
ETNIA
FROM experimentos_comercial.et_csv
""")

<h4>Com a tabela de apoio criada, vamos juntá-la ao nosso data frame principal:</h4>

In [0]:
df_dados = df_dados.join(df_apoio_etnia, (df_dados.TP_COR_RACA == df_apoio_etnia.COD_ETNIA), "left")

In [0]:
df_dados.limit(10).display()

NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_COR_RACA,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_REDACAO,SOMA_NOTA,MÉDIA,COD_ETNIA,ETNIA
200003096603,2020,3,F,2,2,1,0,5300108,Brasília,53,DF,2,1,1,5300108,Brasília,53,DF,1,1,1,1,492.20,556.10,619.70,473.50,1,1,720.00,2861.50,572.30,2,Preta
200002881207,2020,2,F,2,2,1,0,null,null,null,null,null,null,null,2103000,Caxias,21,MA,1,1,1,1,430.50,496.50,482.70,624.30,1,1,380.00,2414.00,482.80,2,Preta
200006376026,2020,4,M,3,1,null,0,null,null,null,null,null,null,null,3300605,Bom Jesus do Itabapoana,33,RJ,0,0,0,0,null,null,null,null,0,null,null,null,null,3,Parda
200001542164,2020,5,M,1,2,1,0,null,null,null,null,null,null,null,4119152,Pinhais,41,PR,0,0,0,0,null,null,null,null,1,null,null,null,null,1,Branca
200005249728,2020,5,F,3,1,null,0,null,null,null,null,null,null,null,1200203,Cruzeiro do Sul,12,AC,1,1,1,1,455.30,542.90,519.60,479.90,1,1,920.00,2917.70,583.54,3,Parda
200002534827,2020,6,M,3,1,null,0,null,null,null,null,null,null,null,2930105,Senhor do Bonfim,29,BA,0,0,0,0,null,null,null,null,0,null,null,null,null,3,Parda
200001948951,2020,4,F,3,1,null,0,null,null,null,null,null,null,null,2516201,Sousa,25,PB,1,1,1,1,539.70,548.20,547.70,479.00,1,1,600.00,2714.60,542.92,3,Parda
200002150830,2020,5,M,2,1,null,0,null,null,null,null,null,null,null,2507507,João Pessoa,25,PB,1,1,1,1,435.30,570.60,542.50,509.50,1,1,680.00,2737.90,547.58,2,Preta
200004093029,2020,3,F,1,1,null,0,null,null,null,null,null,null,null,3552205,Sorocaba,35,SP,1,1,1,1,682.20,719.70,616.30,699.60,0,1,860.00,3577.80,715.56,1,Branca
200001100474,2020,4,F,1,1,null,0,null,null,null,null,null,null,null,2921302,Milagres,29,BA,1,1,1,1,575.70,498.40,503.60,519.40,1,1,800.00,2897.10,579.42,1,Branca


#<h4>Finalizando a modelagem dos dados. Salvar como tabela para iniciarmos o processo de visualização no Power BI.</h4>

In [0]:
df_dados.write.format("parquet").saveAsTable("experimentos_comercial.estudo_base_pb", mode="overwrite")

<h4>Irei descrever o processo de obtenção dos dados no power bi.</h4>
<ul>
<li>Buscar a base no power bi, já está lincado com o databricks.</li>
<li>Nesse caso, como não há updates, irei utilizar a forma de conexão Importar, porém no trabalho, uso a opção DirectQuery, pois uso schedule do databricks para sempre me trazer os dados atualizados.</li>
<li>Analisar no power query, e transformar ou excluir algumas caso necessário.</li>
</ul>